In [61]:
import requests
import pprint
import pandas as pd
import imaplib
import email

## Gmail Client

In [121]:
import talenttrove.app.email.gmail 
import importlib
importlib.reload(talenttrove.app.email.gmail)
from talenttrove.app.email.gmail import Gmail

In [122]:
username ="agarwalpratham2001@gmail.com"

#generated app password
app_password= "xyz"
client = Gmail(username, app_password)
client.authenticate()

In [123]:
from_date = (datetime.now() - timedelta(days=1)).strftime("%d-%b-%Y")  # 7 days ago
to_date = (datetime.now()).strftime("%d-%b-%Y")  # Today
ids = client.get_email_by_date(from_date=from_date, to_date=to_date)
data = client.parse_emails(ids)

x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x
x


In [ ]:
data

In [120]:
client.close()

In [64]:
import http.cookies
cookie_str="""_fbp=fb.1.1694943938226.514835020; user.id=eyJfcmFpbHMiOnsibWVzc2FnZSI6Ik16ZzJNRFE9IiwiZXhwIjpudWxsLCJwdXIiOiJjb29raWUudXNlci5pZCJ9fQ%3D%3D--52ae7b5f23c3b774776a63e18d12439744528751; _hjSessionUser_1996783=eyJpZCI6Ijc0Nzk0Y2UwLWE4MTUtNWVhOC1iMDAxLTg1NmQyYTU2OWVjZCIsImNyZWF0ZWQiOjE2OTQ5NDQwMTE0MjksImV4aXN0aW5nIjp0cnVlfQ==; _gid=GA1.2.1688672101.1696942457; _lr_uf_-hoyxj9=97d909d9-820b-4748-8581-51a0150e803c; _lr_tabs_-hoyxj9%2Fnodeflair={%22sessionID%22:0%2C%22recordingID%22:%225-5dbb285d-e594-4c76-b3ef-89ae6c813be1%22%2C%22webViewID%22:null%2C%22lastActivity%22:1696942545714}; _lr_hb_-hoyxj9%2Fnodeflair={%22heartbeat%22:1696942545714}; cf_clearance=SRqUHV0_JaaYnxWUWGdwUY4agtH6lhcb4wbziRx_v20-1697001805-0-1-8d575e66.bf525c38.9e655532-0.2.1697001805; _hjSession_1996783=eyJpZCI6IjFmMDEyZTQ2LWQyMzItNGM5Ny04MjA5LTlkNmM4OGQ5Mjk5OCIsImNyZWF0ZWQiOjE2OTcwMDE4Nzg3NDYsImluU2FtcGxlIjpmYWxzZSwic2Vzc2lvbml6ZXJCZXRhRW5hYmxlZCI6ZmFsc2V9; _nodeflair_session=SgQmOwi2%2FsrWTlqvMKC6Rr%2F1C2mPiIZdQBm%2B5aWyyHMdth6kujmWtaNchOpglyULeo1nOufxew%2FdRULkNOs%2Bbwk%2BjPz3VQMFlI0gO5sXT0XfNvJE3ZpqzBHa7AF%2B%2BoGcx%2BQ5zOgaqtmd9kr5vAMDO%2FgUC2qOf3rOj9TxOEDwS21XzISVWahxgQFJEhfVmrug%2BGze3lGy1HEatiPfmGwF86v2KEnFdMZhLGZQXw8SuaEhYVih5%2BYeX01zEy3GG1imgM2t2ocEP4MZn2rLQy1F81dsvbacklBlM5FU2LS5SoS%2F6M2qmKT%2FgkbftLeLYlLN0ByWdj9wyugOotZmOUlJB%2FXQ96iiS6%2FDmgqioJKsN8Pin9ZM%2Bu8meoCwqsHSS3gHeiphGx4wHSbZJL%2BEoKJkObmymhOlc4r3kfrPaS2mfhNqVtcW51Bn40n52bYqpzrNkplvu28FbfKaeX04PWCIwpXVf6lOAL1YVCcZ%2BHZWpdUdPPEXGsVwDES4cDnEqYKPFfTuQDELFUTfbrYf9pODWcts3YrEUA%3D%3D--Ij%2FDf1fLhAY1aizO--kFllUCg2W%2FJOPpgiSX84jQ%3D%3D; _ga_0D09HRRGVH=GS1.1.1697001878.8.1.1697002556.60.0.0; _ga=GA1.2.334756147.1694943938; _gat_gtag_UA_116290073_1=1; _hjIncludedInSessionSample_1996783=0"""
cookie = http.cookies.SimpleCookie()
cookie.load(cookie_str)
cookie_dict = {k: v.value for k, v in cookie.items()}
print(cookie_dict)

{'_fbp': 'fb.1.1694943938226.514835020', 'user.id': 'eyJfcmFpbHMiOnsibWVzc2FnZSI6Ik16ZzJNRFE9IiwiZXhwIjpudWxsLCJwdXIiOiJjb29raWUudXNlci5pZCJ9fQ%3D%3D--52ae7b5f23c3b774776a63e18d12439744528751', '_hjSessionUser_1996783': 'eyJpZCI6Ijc0Nzk0Y2UwLWE4MTUtNWVhOC1iMDAxLTg1NmQyYTU2OWVjZCIsImNyZWF0ZWQiOjE2OTQ5NDQwMTE0MjksImV4aXN0aW5nIjp0cnVlfQ==', '_gid': 'GA1.2.1688672101.1696942457', '_lr_uf_-hoyxj9': '97d909d9-820b-4748-8581-51a0150e803c', '_lr_tabs_-hoyxj9%2Fnodeflair': '{%22sessionID%22:0%2C%22recordingID%22:%225-5dbb285d-e594-4c76-b3ef-89ae6c813be1%22%2C%22webViewID%22:null%2C%22lastActivity%22:1696942545714}', '_lr_hb_-hoyxj9%2Fnodeflair': '{%22heartbeat%22:1696942545714}', 'cf_clearance': 'SRqUHV0_JaaYnxWUWGdwUY4agtH6lhcb4wbziRx_v20-1697001805-0-1-8d575e66.bf525c38.9e655532-0.2.1697001805', '_hjSession_1996783': 'eyJpZCI6IjFmMDEyZTQ2LWQyMzItNGM5Ny04MjA5LTlkNmM4OGQ5Mjk5OCIsImNyZWF0ZWQiOjE2OTcwMDE4Nzg3NDYsImluU2FtcGxlIjpmYWxzZSwic2Vzc2lvbml6ZXJCZXRhRW5hYmxlZCI6ZmFsc2V9', '_nodeflair_sessio

In [14]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "sec-fetch-mode": "cors",
    # "If-None-Match": 'W/"1297c0623cd0e995397a9f6796e346c4"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "macOS",
    "Sec-Fetch-Dest": "empty",

}
params={
    "query": None,
    "page": 2,
    "sort_by": "recent",
    "countries[]": "Singapore",
}
response = requests.get('https://nodeflair.com/api/v2/jobs',headers=headers,params=params)

In [15]:
response.status_code

403

In [37]:
response = requests.get('https://nodeflair.com/api/v2/jobs/172520',headers=headers)

In [51]:
response = requests.get("https://api.simplify.jobs/v2/job-list/",headers=headers)

In [52]:
response.json()

{'total': 21,
 'items': [{'title': 'Internships in Human Resources ',
   'description': "Looking to gain practical experience in the field of Human Resources? Look no further than our curated list of internships in Human Resources! At Simplify, we have carefully selected a range of internships from reputable companies across various industries. These internships are ideal for current undergraduate students or recent graduates who are eager to kickstart their careers in Human Resources.\n\nWith our user-friendly search and filter options, you can easily find internships that align with your interests, preferred location, and desired duration. Don't miss out on the opportunity to gain practical skills and industry knowledge while working alongside experienced HR professionals. Explore our list of internships in Human Resources today and take the first step towards a successful HR career!",
   'type': 2,
   'private': False,
   'internship': True,
   'full_time': False,
   'id': '46dc0ee1

In [67]:
tech_response = requests.get("https://api.simplify.jobs/v2/company/?page=1&size=100&value=&workflow_completed=true",headers=headers,cookies=cookie_dict)

In [68]:
tech_response.json()

{'total': 1575,
 'items': [{'name': 'Arc Technologies',
   'id': 'a0a112d0-6df3-45c4-b613-902bf5f6bda4',
   'logo': 'https://storage.googleapis.com/simplify-imgs/companies/a0a112d0-6df3-45c4-b613-902bf5f6bda4/logo.png',
   'description': "Arc's mission is to provide a better way for software startups to raise capital without incurring costly dilution and risky debt. The company has built a comprehensive digital platform  with technology-driven financing that helps remove the biases inherent in conservative financial services – giving SaaS startups access to the capital necessary for operationalizing their success without debt or dilution.",
   'short_description': 'Full-service finance platform for startups',
   'url': 'https://www.arc.tech/',
   'twitter': 'https://twitter.com/join_arc',
   'crunchbase': 'https://www.crunchbase.com/organization/arc-technologies-245d',
   'linkedin': 'https://linkedin.com/company/arc-tech-inc',
   'year_founded': 2021,
   'company_size': 3,
   'benefit

In [73]:
from talenttrove.scrapers.utils import HEADERS
import urllib
# TECH_COMPANIES = "https://nodeflair.com/blog/top-tech-companies-in-singapore"
# response = urllib3.(TECH_COMPANIES,headers=HEADERS,cookies=cookie_dict)

In [76]:
response = urllib.request.urlopen(TECH_COMPANIES)

HTTPError: HTTP Error 403: Forbidden

## Shortlisting Companies

- https://www.timesbusinessdirectory.com/company-listings?page=9
- https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset
- https://beta.data.gov.sg/collections/2/view

In [56]:
company_df = pd.read_csv("output/companies_sorted.csv")

In [62]:
company_df=company_df[company_df['industry'] == 'information technology and services']
company_df = company_df[~company_df['size range'].isin(['1 - 10','11 - 50','51 - 200'])]
company_df.drop(columns=['Unnamed: 0','locality']).reset_index(drop=True).to_csv('talenttrove/data/companies/tech_companies.csv')

In [260]:
import importlib
import talenttrove.scrapers.datagov
importlib.reload(talenttrove.scrapers.datagov)
from talenttrove.scrapers.datagov import DataGovCollection

In [261]:
collection = DataGovCollection(2)

In [194]:
sing_data = collection.collect()

Extracting 27 datsets


 15%|█▍        | 4/27 [00:17<01:22,  3.60s/it]/Users/pratham/Desktop/trademaster/TalentTrove/talenttrove/scrapers/datagov.py:20: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(response.json()['url'])
 52%|█████▏    | 14/27 [02:37<03:01, 13.94s/it]/Users/pratham/Desktop/trademaster/TalentTrove/talenttrove/scrapers/datagov.py:20: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(response.json()['url'])
 56%|█████▌    | 15/27 [03:34<05:24, 27.02s/it]/Users/pratham/Desktop/trademaster/TalentTrove/talenttrove/scrapers/datagov.py:20: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(response.json()['url'])
100%|██████████| 27/27 [06:57<00:00, 15.45s/it]


In [262]:
filter_data = collection.filter(sing_data)

In [263]:
filter_data.shape

(583624, 8)

In [264]:
filter_data.to_csv('talenttrove/data/companies/singapore_companies.csv',index=False)

## Glassdoor

In [17]:
params=[
    {
        "operationName": "JobSearchResultsQuery",
        "variables": {
            "excludeJobListingIds": [],
            "filterParams": [
                {
                    "filterKey": "fromAge",
                    "values": "1"
                }
            ],
            "keyword": "",
            "locationId": 3235921,
            "locationType": "CITY",
            "numJobsToShow": 20,
            "originalPageUrl": "https://www.glassdoor.sg/Job/singapore-singapore-jobs-SRCH_IL.0,19_IC3235921.htm",
            "parameterUrlInput": "IL.0,19_IC3235921",
            "seoFriendlyUrlInput": "singapore-singapore-jobs",
            "seoUrl": True
        },
        "query": "query JobSearchResultsQuery($excludeJobListingIds: [Long!], $keyword: String, $locationId: Int, $locationType: LocationTypeEnum, $numJobsToShow: Int!, $pageCursor: String, $pageNumber: Int, $filterParams: [FilterParams], $originalPageUrl: String, $seoFriendlyUrlInput: String, $parameterUrlInput: String, $seoUrl: Boolean) {\n  jobListings(\n    contextHolder: {searchParams: {excludeJobListingIds: $excludeJobListingIds, keyword: $keyword, locationId: $locationId, locationType: $locationType, numPerPage: $numJobsToShow, pageCursor: $pageCursor, pageNumber: $pageNumber, filterParams: $filterParams, originalPageUrl: $originalPageUrl, seoFriendlyUrlInput: $seoFriendlyUrlInput, parameterUrlInput: $parameterUrlInput, seoUrl: $seoUrl, searchType: SR}}\n  ) {\n    filterOptions\n    indeedCtk\n    jobListings {\n      ...JobView\n      __typename\n    }\n    jobListingSeoLinks {\n      linkItems {\n        position\n        url\n        __typename\n      }\n      __typename\n    }\n    jobSearchTrackingKey\n    jobsPageSeoData {\n      pageMetaDescription\n      pageTitle\n      __typename\n    }\n    paginationCursors {\n      cursor\n      pageNumber\n      __typename\n    }\n    indexablePageForSeo\n    searchResultsMetadata {\n      searchCriteria {\n        implicitLocation {\n          id\n          localizedDisplayName\n          type\n          __typename\n        }\n        keyword\n        location {\n          id\n          shortName\n          localizedShortName\n          localizedDisplayName\n          type\n          __typename\n        }\n        __typename\n      }\n      footerVO {\n        countryMenu {\n          childNavigationLinks {\n            id\n            link\n            textKey\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      helpCenterDomain\n      helpCenterLocale\n      jobAlert {\n        jobAlertExists\n        __typename\n      }\n      jobSerpFaq {\n        questions {\n          answer\n          question\n          __typename\n        }\n        __typename\n      }\n      jobSerpJobOutlook {\n        occupation\n        paragraph\n        __typename\n      }\n      showMachineReadableJobs\n      __typename\n    }\n    serpSeoLinksVO {\n      relatedJobTitlesResults\n      searchedJobTitle\n      searchedKeyword\n      searchedLocationIdAsString\n      searchedLocationSeoName\n      searchedLocationType\n      topCityIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerNameResults\n      topOccupationResults\n      __typename\n    }\n    totalJobsCount\n    __typename\n  }\n}\n\nfragment JobView on JobListingSearchResult {\n  jobview {\n    header {\n      adOrderId\n      advertiserType\n      adOrderSponsorshipLevel\n      ageInDays\n      divisionEmployerName\n      easyApply\n      employer {\n        id\n        name\n        shortName\n        __typename\n      }\n      employerNameFromSearch\n      goc\n      gocConfidence\n      gocId\n      jobCountryId\n      jobLink\n      jobResultTrackingKey\n      jobTitleText\n      locationName\n      needsCommission\n      payCurrency\n      payPeriod\n      payPeriodAdjustedPay {\n        p10\n        p50\n        p90\n        __typename\n      }\n      rating\n      salarySource\n      savedJobId\n      sponsored\n      __typename\n    }\n    job {\n      importConfigId\n      jobTitleId\n      jobTitleText\n      listingId\n      __typename\n    }\n    jobListingAdminDetails {\n      cpcVal\n      importConfigId\n      jobListingId\n      jobSourceId\n      userEligibleForAdminJobDetails\n      __typename\n    }\n    overview {\n      shortName\n      squareLogoUrl\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }
]
response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params)

In [24]:
response.json()[0]['data']['jobListings']

{'filterOptions': '[{"type":"INDUSTRY_N","rank":9,"primary":false,"disabled":false,"name":"industryNId","defLabel":"search-jobs.industry-option.ALL","defValue":-1,"options":{"-1":"search-jobs.industry-id-options.ALL","10026":"Human Resources \\u0026 Staffing","10013":"Information Technology","10011":"Government \\u0026 Public Administration","10016":"Media \\u0026 Communication","10005":"Pharmaceutical \\u0026 Biotechnology","10019":"Energy, Mining, Utilities","10022":"Retail \\u0026 Wholesale","10009":"Education","10012":"Healthcare","10014":"Insurance","10023":"Telecommunications","10006":"Management \\u0026 Consulting","10010":"Finance","10015":"Manufacturing","10004":"Arts, Entertainment \\u0026 Recreation"},"counts":{"10026":127,"10013":12,"10011":11,"10016":5,"10005":5,"10019":3,"10022":3,"10009":3,"10012":3,"10014":3,"10023":2,"10006":2,"10010":2,"10015":2,"10004":1},"sortedKeys":[-1,10004,10009,10019,10010,10011,10012,10026,10013,10014,10006,10015,10016,10005,10022,10023],"sele

In [30]:
import http.cookies
cookie_str="""__Host-next-auth.csrf-token=198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723; _ga=GA1.1.81457931.1696941213; __Secure-next-auth.callback-url=https%3A%2F%2Fwww.carbosjobs.com%2F; _ga_JVWQGY8Q4P=GS1.1.1696959406.2.1.1696959543.0.0.0; __Secure-next-auth.session-token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw; ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog=%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%22%2C1696959402729%5D%2C%22%24session_recording_enabled_server_side%22%3Atrue%2C%22%24console_log_recording_enabled_server_side%22%3Atrue%2C%22%24session_recording_recorder_version_server_side%22%3A%22v2%22%2C%22%24autocapture_disabled_server_side%22%3Afalse%2C%22%24active_feature_flags%22%3A%5B%5D%2C%22%24enabled_feature_flags%22%3A%7B%7D%2C%22%24feature_flag_payloads%22%3A%7B%7D%2C%22%24user_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24stored_person_properties%22%3A%7B%22email%22%3A%22agarwalpratham2001%40gmail.com%22%2C%22name%22%3A%22Pratham%20Agarwala%22%7D%7D"""
cookie = http.cookies.SimpleCookie()
cookie.load(cookie_str)
cookie_dict = {k: v.value for k, v in cookie.items()}
print(cookie_dict)

{'__Host-next-auth.csrf-token': '198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723', '_ga': 'GA1.1.81457931.1696941213', '__Secure-next-auth.callback-url': 'https%3A%2F%2Fwww.carbosjobs.com%2F', '_ga_JVWQGY8Q4P': 'GS1.1.1696959406.2.1.1696959543.0.0.0', '__Secure-next-auth.session-token': 'eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw', 'ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog': '%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%

In [42]:
response = requests.get("https://www.carbosjobs.com/api/job-postings/search?FTSQuery=%28%27new+graduate%27+%7C+%27new+grad%27+%7C+%27entry%27+%7C+%27junior%27+%7C+%27campus%27+%7C+%27early+career%27+%7C+%27university%27+%7C+%27college+grad%27+%7C+%27entry-level%27%29+%26+%28%27trading%27+%7C+%27quantitative%27+%7C+%27market%27+%7C+%27quant%27+%7C+%27trader%27%29+%26+%28%21%272023%27+%26+%21%27fall%27+%26+%21%27winter%27+%26+%21%27m%2Ff%2Fd%27+%26+%21%27senior%27+%26+%21%27intern%27+%26+%21%27internship%27+%26+%21%27internships%27+%26+%21%27data+entry%27%29&filterName=Quant+%26+Trading%2C+New+Grad+%28USA%29",headers=headers,cookies=cookie_dict)

In [43]:
import json
len(json.loads(response.json()))

28

In [ ]:
https://www.google.com/search?q=google+jobs&oq=google+jobs&sourceid=chrome&ie=UTF-8&ibp=htl;jobs&sa=X&ved=2ahUKEwifttedluaBAxWpxDgGHdNwCRQQutcGKAF6BAgOEAY&sxsrf=AM9HkKlkNKxI2yk2kPylgS6nYtrhsl31Lg:1696758567924#htivxrt=jobs&fpstate=tldetail&htichips=date_posted:3days&htischips=date_posted;3days&htidocid=UbM2CbLbvu4OMmqWAAAAAA%3D%3D

In [ ]:
https://www.linkedin.com/jobs/search/?f_C=1009%2C155134%2C9263339%2C3634137&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3703127521%2C3693596011%2C3693593672%2C3727737691%2C3705389269%2C3705383975%2C3715841313%2C3705385738%2C3705649976&lipi=urn%3Ali%3Apage%3Acompanies_company_jobs_jobs%3Bb802d511-1d27-453d-a8d8-51463c7c7316

## Carbos

In [52]:
import importlib
import talenttrove.scrapers.carbos
importlib.reload(talenttrove.scrapers.carbos)
from talenttrove.scrapers.carbos import Carbos,ALL_ENGINEERING_GRAD,ALL_ENGINEERING_INTERNSHIPS,ALL_QUANT_TRADING_INTERNSHIPS,ALL_QUANT_TRADING_GRAD

In [46]:
cookie_str="""__Host-next-auth.csrf-token=198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723; _ga=GA1.1.81457931.1696941213; __Secure-next-auth.callback-url=https%3A%2F%2Fwww.carbosjobs.com%2F; _ga_JVWQGY8Q4P=GS1.1.1696959406.2.1.1696959543.0.0.0; __Secure-next-auth.session-token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw; ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog=%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%22%2C1696959402729%5D%2C%22%24session_recording_enabled_server_side%22%3Atrue%2C%22%24console_log_recording_enabled_server_side%22%3Atrue%2C%22%24session_recording_recorder_version_server_side%22%3A%22v2%22%2C%22%24autocapture_disabled_server_side%22%3Afalse%2C%22%24active_feature_flags%22%3A%5B%5D%2C%22%24enabled_feature_flags%22%3A%7B%7D%2C%22%24feature_flag_payloads%22%3A%7B%7D%2C%22%24user_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24stored_person_properties%22%3A%7B%22email%22%3A%22agarwalpratham2001%40gmail.com%22%2C%22name%22%3A%22Pratham%20Agarwala%22%7D%7D"""
client = Carbos(cookie_str)

In [53]:
jobs_1 = client.collect(ALL_ENGINEERING_GRAD)
jobs_1 = pd.DataFrame(jobs_1)
jobs_2 = client.collect(ALL_ENGINEERING_INTERNSHIPS)
jobs_2 = pd.DataFrame(jobs_2)
jobs_3 = client.collect(ALL_QUANT_TRADING_INTERNSHIPS)
jobs_3 = pd.DataFrame(jobs_3)
jobs_4 = client.collect(ALL_QUANT_TRADING_GRAD)
jobs_4 = pd.DataFrame(jobs_4)

Extracted 278 jobs
Extracted 569 jobs
Extracted 101 jobs
Extracted 28 jobs


In [59]:
pd.concat([jobs_1,jobs_2,jobs_3,jobs_4],axis=0).to_csv("talenttrove/data/jobs/carbos.csv")

## My CarrersFuture

In [93]:
import importlib
import talenttrove.scrapers.mycareersfuture
importlib.reload(talenttrove.scrapers.mycareersfuture)
from talenttrove.scrapers.mycareersfuture import MyCareersFuture

In [96]:
client = MyCareersFuture()

In [97]:
data = client.collect(sleep_time=2)

Total pages 67


100%|██████████| 66/66 [02:57<00:00,  2.68s/it]

Extracted 6674 jobs


In [102]:
pd.DataFrame(data).to_json("talenttrove/data/jobs/mycareersfuture.json",orient="records")

## Simplify Company and Job Scraper

In [10]:
import importlib
import talenttrove.scrapers.simplifyJobs
importlib.reload(talenttrove.scrapers.simplifyJobs)
from talenttrove.scrapers.simplifyJobs import SimplifyJobs

In [11]:
from talenttrove.scrapers.utils import HEADERS

In [62]:
response=requests.get(url=f"https://api.simplify.jobs/v2/company?size=100&page=1",headers=HEADERS)

In [64]:
response.text

'\n<html><head>\n<meta http-equiv="content-type" content="text/html;charset=utf-8">\n<title>500 Server Error</title>\n</head>\n<body text="#000000" bgcolor="#ffffff">\n<h1>Error: Server Error</h1>\n<h2>The server encountered an error and could not complete your request.<p>Please try again in 30 seconds.</h2>\n<h2></h2>\n<script defer src="https://static.cloudflareinsights.com/beacon.min.js/v8b253dfea2ab4077af8c6f58422dfbfd1689876627854" integrity="sha512-bjgnUKX4azu3dLTVtie9u6TKqgx29RBwfj3QXYt5EKfWM/9hPSAI/4qcV5NACjwAo8UtTeWefx6Zq5PHcMm7Tg==" data-cf-beacon=\'{"rayId":"810f36e81e2d8802","version":"2023.8.0","r":1,"b":1,"token":"52b7a31841144c1eb06bf4f6e182d841","si":100}\' crossorigin="anonymous"></script>\n</body></html>\n'

In [60]:
client= SimplifyJobs()
jobs = client.collect()

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

## Techjob Repo Scrapper
- https://techjobrepo.com/

In [1]:
import importlib
import talenttrove.scrapers.techjobrepo
importlib.reload(talenttrove.scrapers.techjobrepo)
from talenttrove.scrapers.techjobrepo import TechJobRepo

In [2]:
client= TechJobRepo()
jobs = client.collect()

Extracted 289 jobs


In [3]:
import pandas as pd

In [7]:
pd.json_normalize(jobs).to_csv("talenttrove/data/techjobrepo.csv",index=False)